In [1]:
import os 
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [9]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import gradio as gr

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAI

In [14]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=GOOGLE_API_KEY)

In [19]:
llm2 = GoogleGenerativeAI(model="gemini-2.0-flash",api_key=GOOGLE_API_KEY)

In [16]:
llm.invoke("Hii").content

'Hi there! How can I help you today?'

In [21]:
llm2.invoke("Hii")

'Hi there! How can I help you today?'

In [22]:
def process_doc(file_pats):
    documents=[]
    
    for path in file_pats:
        if path.endswith(".pdf"):
            loader = PyPDFLoader(path)
        elif path.endswith(".txt"):
            loader = TextLoader(file_pats)
        else:
            continue
        
        documents.extend(loader.load())
        
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_documents(documents)
    
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.getenv("GOOGLE_API_KEY"))
    vector_store = FAISS.from_documents(chunks, embeddings)
    
    return vector_store

In [24]:
document_paths = [os.path.join("documents", f) for f in os.listdir("documents")]
vector_store = process_doc(document_paths)

In [25]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/jl/y_70cc5d1mqd7h_yq2qh75dw0000gn/T/ipykernel_10514/3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [26]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    memory=memory
)

In [27]:
def generate_response(message, history):
    response = conversation_chain({"question": message})
    return response["answer"]

In [28]:
demo = gr.ChatInterface(
    fn=generate_response,
    title="Your AI Knowledge Assistant",
    description="Ask me anything about your documents!",
    theme="soft"
)


/Users/sidhya/Desktop/Development/chatbot/chatbot/lib/python3.12/site-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
